In [1]:
pip install numpy pandas matplotlib tensorflow scikit-learn


In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# Load the dataset
data = pd.read_csv('stock_prices.csv', date_parser=True)

# Use only 'Close' prices for prediction
data = data[['Date', 'Close']]
data.set_index('Date', inplace=True)

# Normalize the 'Close' prices
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data[['Close']])

# Prepare the dataset for training
def create_dataset(data, time_step=1):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

time_step = 60
X, y = create_dataset(scaled_data, time_step)

# Reshape X to be [samples, time steps, features]
X = X.reshape((X.shape[0], X.shape[1], 1))


FileNotFoundError: [Errno 2] No such file or directory: 'stock_prices.csv'

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Function to create and compile the model
def create_lstm_model(units, dropout_rate=0.2, layers=1):
    model = Sequential()
    model.add(LSTM(units, return_sequences=(layers > 1), input_shape=(time_step, 1)))

    for _ in range(layers - 1):
        model.add(LSTM(units, return_sequences=True))

    model.add(Dropout(dropout_rate))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Parameters to experiment with
units_list = [50, 100]
dropout_rates = [0.2, 0.5]
layers_list = [1, 2]
epochs = 50
batch_size = 32

# Train the model with different configurations
results = {}

for units in units_list:
    for dropout_rate in dropout_rates:
        for layers in layers_list:
            model = create_lstm_model(units, dropout_rate, layers)
            history = model.fit(X, y, epochs=epochs, batch_size=batch_size, verbose=0)
            results[(units, dropout_rate, layers)] = history


NameError: name 'time_step' is not defined